In [1]:
# use a baseline trained only with PPL, which has already converged and will be used as our baseline.
# they do teacher forcing all the way.

In [2]:
from data_generators import get_iterator, get_dataset
from classifiers import theme_classifier

In [3]:
import torch
import torch.nn as nn

In [4]:
from torchtext.vocab import GloVe 
GLOVE_EMBEDDING = GloVe(name="6B", dim=300)

In [5]:
train_dataset, val_dataset, test_dataset, review_text_FIELD, theme_FIELD = get_dataset(vectors = GLOVE_EMBEDDING)

In [124]:
batch_size = 20
train_iter = get_iterator(train_dataset, batch_size, train=True, shuffle=True, repeat=False)
val_iter = get_iterator(val_dataset, batch_size, train=False, shuffle=True, repeat=False)
test_iter = get_iterator(test_dataset, batch_size, train=False, shuffle=True, repeat=False)

In [125]:
val_list = list(val_iter)

In [126]:
batch = val_list[50]
x = batch.review_text.transpose(1, 0).int()[:10]
y = batch.theme.int()

for idx in range(x.shape[0]):
    #print(x.shape, y.shape)
    print("{} | {}".format(' '.join([train_dataset.fields['review_text'].vocab.itos[_] for _ in x[idx]]),
         train_dataset.fields['theme'].vocab.itos[y[idx]]))

<sos> awesome sauce <eos> | other
<sos> totally recommend <eos> | other
<sos> <unk> . <eos> | other
<sos> truly amazing <eos> | other
<sos> talky . <eos> | other
<sos> must see <eos> | other
<sos> 3.5 5 <eos> | other
<sos> 5 stars <eos> | other
<sos> movie ? <eos> | other
<sos> brutal . <eos> | other


In [127]:
theme_FIELD.vocab.itos

['<unk>', 'other', 'plot', 'acting', 'effect', 'production']

In [128]:
vocab_size = review_text_FIELD.vocab.vectors.shape[0]
label_size = len(theme_FIELD.vocab) - 1
emb_dim = review_text_FIELD.vocab.vectors.shape[1]
vectors = train_dataset.fields["review_text"].vocab.vectors
hidden_dim = 1024
layers = 2
dropout = .5

vocab_size, label_size, emb_dim, vectors.shape

(12304, 5, 300, torch.Size([12304, 300]))

In [109]:
class BaseModel(nn.Module):

    def __init__(self, 
                 ninp = vocab_size, 
                 linp = label_size, 
                 emb_dim = emb_dim, 
                 emb_lab = 20,
                 nhid = hidden_dim, 
                 nout = vocab_size, 
                 nlayers = layers, 
                 dropout = dropout, 
                 vectors = vectors,
                 pretrained = False):
        super().__init__()
        
        self.ninp = ninp
        self.linp = linp
        self.emb_dim = emb_dim
        self.emb_lab = emb_lab
        self.nhid = nhid
        self.nout = nout
        self.nlayers = nlayers
        self.drop = nn.Dropout(dropout)

        self.word_embedding = nn.Embedding(ninp, emb_dim)
        self.label_embedding = nn.Embedding(linp, emb_lab)
        
        self.rnn = nn.LSTM(emb_dim + emb_lab, nhid, nlayers, dropout=dropout)
        self.rnn.flatten_parameters()
        self.decoder = nn.Linear(nhid, nout)

        if pretrained:
            self.word_embedding.weight.data.copy_(vectors)
            self.word_embedding.from_pretrained(GLOVE_EMBEDDING.vectors)
            
        self.init_weights()    
            
    def init_weights(self):
        initrange = .1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, reviews, labels, hidden):
        R = self.word_embedding(reviews)
        L = self.label_embedding(labels)
        L = torch.cat([L.unsqueeze(0)]*R.shape[0])
        X = torch.cat([R, L], -1)
        
        X = self.drop(X)
        X, hidden = self.rnn(X, hidden)
        X = self.drop(X)
        
        X = self.decoder(X)
        return X, hidden
      

In [46]:
def repackage_hidden(h):
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [47]:
import numpy as np
import gc

In [48]:
EOS_token = review_text_FIELD.vocab.stoi['<eos>']
EOS_token

3

In [262]:
def forward_pass(model, data, targets, labels, split_tf):
    
    data_tf, targets_tf = data[:split_tf,:],  targets[:split_tf,:]
    data_nf, targets_nf = data[split_tf:,:],  targets[split_tf:,:]

    output_flat = None
    hidden = None
    
    if split_tf > 0:

        data = data_tf
        output_tf, hidden = model(data, labels, hidden)
        repackage_hidden(hidden)
        output_flat = output_tf.contiguous().view(-1, vocab_size)

    if split_tf < targets.shape[0]:

        data = data_nf
        shape = tuple((*targets_nf.shape, vocab_size))
        output_nf = torch.zeros(shape).cuda()
        hidden_i = None
        data_i = data[0,:]    

        for di in range(targets_nf.shape[0]):
            params = data_i.unsqueeze(0), labels, hidden_i
            output_i, hidden_i = model(*params)
            hidden_i = repackage_hidden(hidden_i)
            topv, topi = output_i.topk(1)
            data_i = topi.squeeze().detach()
            output_nf[di,:] = output_i

        temp_output_flat = output_nf.contiguous().view(-1, vocab_size)
        if output_flat is None:
            output_flat = temp_output_flat
        else:
            output_flat = torch.cat([output_flat, temp_output_flat], 0)
            
    return output_flat

In [263]:
def evaluate(model, data_source, criterion, split_tf = 10000):
    model.eval()
    total_loss_e = 0
    total_number_of_words = 0
    
    with torch.no_grad():
        for i, batch in enumerate(data_source):
            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            
            if batch.shape[0] > 3:
                data, targets = batch[1:-1,:], batch[2:,:]
                target_flat = targets.contiguous().view(-1)

                output_flat = forward_pass(model, data, targets, labels, split_tf)
                
                batch_loss = criterion(output_flat, target_flat).detach().item()
                number_of_words = data.shape[0] * data.shape[1]
                total_loss_e += batch_loss * number_of_words
                total_number_of_words += number_of_words
            
    return total_loss_e / total_number_of_words

In [264]:
val_list[-20].review_text.shape

torch.Size([109, 20])

In [267]:
evaluate(model, val_list[-20:], criterion, 200)

5.138088450335147

In [266]:
evaluate(model, val_list[-20:], criterion, 80)

6.674701739050949

In [15]:
def train(model, ep0, epN, train_iter, dev_iter, optimizer, criterion, 
          max_grad_norm, model_name, best_ppl = float('inf')):
    
    best_ppl = best_ppl
    
    len_train_iter = len(train_iter)
    for epoch in range(ep0, epN):
        model.train()
        total_loss_e = 0
        total_number_of_words = 0 
        
        for i, batch in enumerate(train_iter):

            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            hidden = None
            if batch.shape[0] > 3:
                data, targets = batch[1:-1,:], batch[2:,:]
                
                
                output, hidden = model(data, labels, hidden)
                repackage_hidden(hidden)

                output_flat = output.contiguous().view(-1, vocab_size)
                target_flat = targets.contiguous().view(-1)
                batch_loss = criterion(output_flat, target_flat)
                
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                optimizer.zero_grad()
                batch_loss.backward()
                optimizer.step()

                number_of_words = data.shape[0] * data.shape[1]
                total_loss_e += batch_loss.detach().item() * number_of_words
                total_number_of_words += number_of_words
            
                
                if i % 500 == 0:
                    cur_loss = batch_loss.detach().item() #np.mean(total_loss_e)#
                    tr_ppl_print = np.exp(cur_loss)
                    print("| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {}".format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3)))

                
                if i % 4999 == 1: #len_train_iter - 1:
                    cur_loss = batch_loss.detach().item() #np.mean(total_loss_e) #
                    tr_ppl_print = np.exp(cur_loss)
                    gc.collect()
                    #torch.cuda.empty_cache()
                    val_loss_eval = evaluate(model, dev_iter, criterion)
                    val_ppl_print = np.exp(val_loss_eval)
                    
                    template = "| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {} | val_loss {} | val_ppl {}"
                    print(template.format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3), 
                            np.round(val_loss_eval, 3), np.round(val_ppl_print, 3)))

                    if val_ppl_print < best_ppl :
                        print('old best ppl {} new best ppl {}'.format(best_ppl, val_ppl_print))
                        best_ppl = val_ppl_print
                        best_model_name = 'best_model_{}_{}.model'.format(model_name, best_ppl)
                        print('save model...', best_model_name)
                        with open(best_model_name, 'wb') as file:
                            torch.save(model, file) 

                    gc.collect()
                    model.train()
                    
                if i == 40000: break

In [110]:
with open('./baseline/best_model_base_model_ppl_67.5461793533482.model', 'rb') as file:
    model = torch.load(file)
    model.eval()

In [111]:
model

BaseModel(
  (drop): Dropout(p=0.5)
  (word_embedding): Embedding(12304, 300)
  (label_embedding): Embedding(5, 20)
  (rnn): LSTM(320, 1024, num_layers=2, dropout=0.5)
  (decoder): Linear(in_features=1024, out_features=12304, bias=True)
)

In [112]:
learning_rate = 0.00001
criterion = nn.CrossEntropyLoss(reduction='mean',
                       ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(.999999, .999999))

In [ ]:
train(model,
      ep0 = 1,
      epN = 2,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 10,
      model_name = 'base_model_ppl',
      best_ppl = 81.12277368937825)